# Lesson 2: Datasets For Reinforcement Learning Training

### Loading and exploring the datasets

"Reinforcement Learning from Human Feedback" **(RLHF)** requires the following datasets:
- Preference dataset
  - Input prompt, candidate response 0, candidate response 1, choice (candidate 0 or 1)
- Prompt dataset
  - Input prompt only, no response

# Conceptual summary: 

- input text: ..... long text
- summary: a short summary

    - The training dataset would be: {input text, summary} pairs. But there is no correct to summarize the texts. There are many valid summaries we can write. The preferences are hard to quantify. 

    - Introduce supervised finue tuning: {input text, summary}

    - RLHF: the training dataset would be: {input text, summary 1, summary 2, human preference.} human preference is labeled by humans, like which summary they like better... time/hr-consuming. 
    
Three stages: 
    1. preference dataset --> 
    2. training (supervised) --> 
    3. reward model (in an reinforcement learning training loop) to fine tune our "base LLM". 
  
Stage 1: Preference dataset

    Summarize the following text: I want to satrt gardening but I don't ... --> BASE LLM --> Summary 1 (score 7), summary 2 (score 2)...

    Scales like this are subjective and vary acorss people. Instead, one way of doing this, is to allow people to decide which summary they prefer and state the reasons behind their decisions. The dataset indicates one person's preferences, but not all human beings' preferences in general. 

    What is the point of tuning? more useful? less harmful contents? More positive? ... Need to define the goal first. 


Stage 2: Reward Model

    Itself as another LLM (inference)
    The input would be prompt, completion, and the output is the scalar value indicating how good the completion is. Eseentially, the reward model is a regression model that outputs numbers. 
    So one record of the preference dataset can be seen as {Prompt, winning candidate, losing candidate}. For each candidate completion, we get a score. While training, we are trying to minimize the reward model loss function. 
    
Stage 3: Use RM in an RL loop to fine tune base llm. 

    During inference, we send the {prompt, completion} to a reward model to get a scalar indicating how good the completion is. The higher the number, the more likely the completion agrees with the human reviewer's assessment. 
    Use the reward modell in an RL loop to fine tune the base LLM. to produce the completion that maximize the scalar output of the reward model. Here we introduce a new dataset: prompt dataset: prompt1, prompt2, prompt3....
    RL is useful when you want to train a model to learn how to solve a task that involves a complex and fairly open-ended objective. You may not know in advance what the optimal solution would be, but you can give the model rewards to guide it to an optimal solution. Formally, it teaches the agent to take actions that maximize the reward. The agent takes action that interacts with the environment and get a state that is different from the environment, and then get a reward that helps the agent learn the rules of the environment. (e.g. alphaGo)
    A possible set of actions along with the probability that each action will lead to a higher reward. This set of actions is called a policy. The goal of RL is to learn a policy that can maximize the reward. The policy = "The brain of the agent". 
    How does RL relate to RLHF?: The policy is the base LLM that we want to tune. The current state is whatever is in the context (such as, the prompt and any generated text un to this point and actions are genertating tokens). Each time the base LLM outputs a completion, it receives a reward from the reward model indicating how aligned that generated text is. Learning the policy that maximizes the reward amounts to a LLM that produces completions with high scores from the reward model. In RLHF, the policy is learned via the policy gradient method, proximal policy optimization or PPO. This is a standard reinforcement learning algorithm.  
    
    
We get a prompt from the prompt dataset, the BASE LLM compelete the prompt and send the completion and the prompt pair to the reward model, and then get a reward score. It then gets the model weights updated via PPO. and back to Base llm. In practice, you even add the penalty term to refrain the tuned model does not stray too far awar from teh base model. 

Source: Deep Reinforcement Learning from human preferences (Christiano etc)
Training language models to follow instructions with human feedback. 


### Full fine tuning

- re-train the base model updating all of the model weights (parameters)

### Parameter efficient fine tuning: 
- retrin the base model but only update a small subset of the model parameters
- keep all the other parameters frozen. 
- deatiled methodologies are under research 

To summarize, the code loads two datasets (here preference and prompt), explores the structure of the preference data which contains prompts and their completions.

#### Preference dataset

In [1]:
preference_dataset_path = 'sample_preference.jsonl'

In [2]:
import json

In [3]:
preference_data = []

In [5]:
with open(preference_dataset_path) as f:
    for line in f:
        preference_data.append(json.loads(line))

- Print out to explore the preference dataset

In [6]:
sample_1 = preference_data[0]

In [7]:
print(type(sample_1))

<class 'dict'>


In [8]:
# This dictionary has four keys
print(sample_1.keys())

dict_keys(['input_text', 'candidate_0', 'candidate_1', 'choice'])


- Key: 'input_test' is a prompt.

In [9]:
sample_1['input_text']

'I live right next to a huge university, and have been applying for a variety of jobs with them through their faceless electronic jobs portal (the "click here to apply for this job" type thing) for a few months. \n\nThe very first job I applied for, I got an interview that went just so-so. But then, I never heard back (I even looked up the number of the person who called me and called her back, left a voicemail, never heard anything).\n\nNow, when I\'m applying for subsequent jobs - is it that same HR person who is seeing all my applications?? Or are they forwarded to the specific departments?\n\nI\'ve applied for five jobs there in the last four months, all the resumes and cover letters tailored for each open position. Is this hurting my chances? I never got another interview there, for any of the positions. [summary]: '

In [10]:
# Try with another examples from the list, and discover that all data end the same way
preference_data[2]['input_text'][-50:]

'plan something in those circumstances. [summary]: '

- Print 'candidate_0' and 'candidate_1', these are the completions for the same prompt.

In [11]:
print(f"candidate_0:\n{sample_1.get('candidate_0')}\n")
print(f"candidate_1:\n{sample_1.get('candidate_1')}\n")

candidate_0:
 When applying through a massive job portal, is just one HR person seeing ALL of them?

candidate_1:
 When applying to many jobs through a single university jobs portal, is just one HR person reading ALL my applications?



- Print 'choice', this is the human labeler's preference for the results completions (candidate_0 and candidate_1)

In [12]:
print(f"choice: {sample_1.get('choice')}")

choice: 1


#### Prompt dataset

In [13]:
prompt_dataset_path = 'sample_prompt.jsonl'

In [14]:
prompt_data = []

In [15]:
with open(prompt_dataset_path) as f:
    for line in f:
        prompt_data.append(json.loads(line))

In [16]:
# Check how many prompts there are in this dataset
len(prompt_data)

6

**Note**: It is important that the prompts in both datasets, the preference and the prompt, come from the same distribution.

For this lesson, all the prompts come from the same dataset of [Reddit posts](https://github.com/openai/summarize-from-feedback).

In [17]:
# Function to print the information in the prompt dataset with a better visualization
def print_d(d):
    for key, val in d.items():
        print(f"key:{key}\nval:{val}\n")

In [18]:
print_d(prompt_data[0])

key:input_text
val:I noticed this the very first day! I took a picture of it to send to one of my friends who is a fellow redditor. Later when I was getting to know my suitemates, I asked them if they ever used reddit, and they showed me the stencil they used to spray that! Along with the lion which is his trademark. 
 But [summary]: 



In [19]:
# Try with another prompt from the list
print_d(prompt_data[1])

# only one key, input_text

key:input_text
val:Nooooooo, I loved my health class! My teacher was amazing! Most days we just went outside and played and the facility allowed it because the health teacher's argument was that teens need to spend time outside everyday and he let us do that. The other days were spent inside with him teaching us how to live a healthy lifestyle. He had guest speakers come in and reach us about nutrition and our final was open book...if we even had a final.... [summary]: 



In [21]:
preference_data[1]["input_text"][-50:]
# it always ends with '[summary]'

"mostly to places that I've never seen? [summary]: "

In [22]:
preference_data[1]["input_text"][-100:]
# it always ends with '[summary]'

"ng to me from my childhood, or should I try to go mostly to places that I've never seen? [summary]: "

In [23]:
preference_data[2]["input_text"][-100:]
# it always ends with '[summary]'

"s holiday)\n\nI feel like I'm going crazy trying to plan something in those circumstances. [summary]: "